# 层和块 Layers and Modules

nn.Sequential容器将多个层按顺序组合

In [22]:
import torch
from torch import nn
# functional: 包含激活函数（如ReLU）、损失函数等函数式操作
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),  # 输入层：20 → 256
                    nn.ReLU(),           # 激活函数
                    nn.Linear(256, 10)   # 输出层：256 → 10
)

X = torch.rand(2, 20)  # 生成形状为(2, 20)的随机输入（批量大小=2，特征数=20）
net(X)         # 前向传播，输出形状为(2, 10)

tensor([[ 0.0683,  0.1392, -0.2081, -0.0678,  0.2420,  0.0115, -0.1843,  0.1858,
         -0.0878, -0.2748],
        [-0.0109,  0.1838, -0.2076,  0.0626,  0.1294,  0.1215, -0.0381,  0.2039,
         -0.0705, -0.1320]], grad_fn=<AddmmBackward0>)

# 1. 自定义块

In [23]:
# 继承 nn.Module 自定义了一个多层感知机（MLP）模型
class MLP(nn.Module):
    def __init__(self):
        super().__init__()                # 初始化父类 nn.Module
        self.hidden = nn.Linear(20, 256) # 定义隐藏层（20 → 256）
        self.out = nn.Linear(256, 10)    # 定义输出层（256 → 10）

    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) # 前向传播逻辑

In [24]:
net = MLP()
net(X)

tensor([[ 0.1144,  0.1652,  0.0131,  0.0139, -0.2122,  0.0924,  0.1146,  0.0536,
          0.1179,  0.0463],
        [ 0.0855,  0.2030, -0.1354,  0.1020, -0.1612,  0.0329,  0.0735, -0.0099,
          0.0357,  0.0965]], grad_fn=<AddmmBackward0>)

# 2. 顺序块

In [15]:
class MySequential(nn.Module):
    # 可以传入任意数量的位置参数（即多个模块）：MySequential(module1, module2, module3) 里面作为args 列表
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 使用自增索引作为键，确保唯一性
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [16]:
net = MySequential(nn.Linear(20, 256),
                   nn.ReLU(),
                   nn.Linear(256, 10))
net(X)

tensor([[-0.0908,  0.1541, -0.0637, -0.0449,  0.0051, -0.0477, -0.1111, -0.3128,
         -0.0575, -0.0918],
        [-0.0187,  0.1894,  0.0662, -0.0227,  0.0449, -0.0736, -0.0416, -0.3771,
         -0.0382, -0.0785]], grad_fn=<AddmmBackward0>)

# 3. 在前向传播函数中执行代码

In [17]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # 固定权重
        self.linear = nn.Linear(20, 20)  # 可学习的全连接层

    def forward(self, X):
        X = self.linear(X)  # 第一次线性变换
        X = F.relu(torch.mm(X, self.rand_weight) + 1)  # 固定权重的线性变换 + ReLU
        X = self.linear(X)  # 第二次线性变换（参数共享）
        while X.abs().sum() > 1:  # 动态规范化
            X /= 2
        return X.sum()  # 输出标量

In [18]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1705, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法

In [20]:
# 通过组合自定义模块和内置层构建了一个嵌套的神经网络模型 chimera
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),
                        nn.Linear(16, 20),
                        FixedHiddenMLP()
                        )
chimera(X)

tensor(-0.0526, grad_fn=<SumBackward0>)